# Verifications for the Zernike branch

## 0. Set up the environment

This notebook compares the Zernike branch and the master (v2.10) branch. These tests should take less than an hour to complete.

### 0.1 Clone and compile
To start, please checkout the Zernike branch and the master branch (__v2.10, not the latest master__) in two different folders. The following git commands clone these two branches into a folder `Zernike` and a folder `master`.

```
git clone -b Zernike https://github.com/PrincetonUniversity/SPEC.git Zernike
git clone -b v2.10 https://github.com/PrincetonUniversity/SPEC.git master
```

After cloning, please compile them seperately as normal.

### 0.2 Dependencies
Before running this notebook, one will need to set up the `py_spec` package. __Please use the package in the Zernike branch, not the master branch or pypi branch!!!__ 

One will need `numpy`, `h5py` and `f90nml` packages. If you haven't installed them, please do so by

`pip install --user numpy h5py f90nml`

Or one can choose to use `conda` to manage the packages.

One can set up the `py_spec` package by the following two means:

1. Setting the environment before starting Jupyter Notebook

`export PYTHONPATH=$PYTHONPATH:/path/to/Zernike/Utilities/pythontools`

2. Setting the environment in this Notebook by copying the following commands into a new cell and running them before anything else in this notebook:

```python
import sys
sys.path.append('/path/to/Zernike/Utilities/pythontools')
```

Then we can import `py_spec` and relevant objects in it.

__Important: check the version of the `py_spec` package. It should be `0.0.1 Zernike` !!!__

In [1]:
import sys
sys.path.append('/home/abaillod/SPEC/Utilities/pythontools')

In [2]:
import py_spec
from py_spec import SPEC
from py_spec.SPECNamelist import SPECNamelist
import os
import numpy as np
print(py_spec.__version__)

0.0.1 Zernike


### 0.3 Specify the path to the input files and the path to the executables, for both v2.10 and the Zernike branch

__Please change the following paths to match your machine.__ Also set up the mpi command.

In [3]:
Zernike_home_folder='/Users/zhisong/Codes/SPEC_hdf5master/SPEC'
Zernike_executable='/Users/zhisong/Codes/SPEC_hdf5master/SPEC/xspec'

master_home_folder='/Users/zhisong/Codes/SPEC_cc/SPEC'
master_executable='/Users/zhisong/Codes/SPEC_cc/SPEC/xspec'

mpirun = 'mpirun -np'
#mpirun = 'mpiexec -n'

### 0.4 Define a few helper functions that runs the cases and compares the results

In [4]:
def run_case(input_file_name, ncpus=1, LradZernike=None, Lradmaster=None, Mpol=None, Ntor=None, others=None):
    
    class Result:
        pass
    
    # For the Zernike branch

    # enter Zernike test directory
    os.chdir(Zernike_home_folder)
    # uses 3 cpus
    Zernike_xspec_command=mpirun + ' {:d} ' + Zernike_executable
    Zernike_xspec_command=Zernike_xspec_command.format(ncpus)

    # read the namelist
    Zernike_namelist = SPECNamelist(input_file_name)
    # compute the force gradient from scratch
    Zernike_namelist['globallist']['LreadGF']=False
    # do not update Bns initially
    Zernike_namelist['numericlist']['LautoinitBn'] = 0
    # do not generate Poincare plots
    Zernike_namelist['diagnosticslist']['nppts']=0
    # replace the Lrad
    if LradZernike is not None:
        Zernike_namelist['physicslist']['Lrad'][0] = LradZernike

    # For the v2.1 branch

    # enter master test directory
    os.chdir(master_home_folder)
    # uses 3 cpus
    master_xspec_command=mpirun + ' {:d} ' + master_executable
    master_xspec_command=master_xspec_command.format(ncpus)

    # read the namelist
    master_namelist = SPECNamelist(input_file_name)
    # compute the force gradient from scratch
    master_namelist['globallist']['LreadGF']=False
    # do not update Bns initially
    master_namelist['numericlist']['LautoinitBn'] = 0
    # do not generate Poincare plots
    master_namelist['diagnosticslist']['nppts']=0
    # replace the Lrad
    if Lradmaster is not None:
        master_namelist['physicslist']['Lrad'][0] = Lradmaster
        
        
    # replace Mpol or Ntor
    if Mpol is not None or Ntor is not None:
        if Ntor is None:
            Ntor = Zernike_namelist['physicslist']['Ntor']
        if Mpol is None:
            Mtor = Zernike_namelist['physicslist']['Ntor']
        Zernike_namelist.update_resolution(Mpol, Ntor)
        master_namelist.update_resolution(Mpol, Ntor)
        
    # replace other namelist items
    if others is not None:
        for item in others:
            Zernike_namelist[item[0]][item[1]]=item[2]
            master_namelist[item[0]][item[1]]=item[2]
        
    # run the cases
    os.chdir(Zernike_home_folder)
    Zernike_output = Zernike_namelist.run(Zernike_xspec_command, force=True)
    os.chdir(master_home_folder)
    master_output = master_namelist.run(master_xspec_command, force=True)

    # define the result object
    result = Result()
    result.Zernike_output = Zernike_output
    result.master_output = master_output
    
    return result

def check_interface(result, key='Rbc', rtol=1e-12):
    # check if Rbc matches
    Zernike_b=np.array(getattr(result.Zernike_output.output, key))
    master_b=np.array(getattr(result.master_output.output, key))
    
    b_diff = np.max(np.abs(Zernike_b - master_b)) 

    if (b_diff < rtol):
        print(key, " matches, diff", b_diff)
    else:
        print(key, " doesn't match, diff", b_diff)
        
def check_field_on_interfaces(result, key='Bp', rtol=1e-12):
    # check if the magnetic field in xyz matches on the interfaces
    Mvol = result.Zernike_output.output.Mvol
    
    diff_B_max = 0.0
    B_max = 0.0
    for lvol in range(Mvol):
        Zernike_B = np.array(getattr(result.Zernike_output.grid, key)[0])
        master_B = np.array(getattr(result.master_output.grid, key)[0])
        # the inner boundary
        if (lvol > 0):
            diff_B = np.abs(Zernike_B[:,0]-master_B[:,0])
            diff_B_max = np.max([diff_B_max, np.max(diff_B)])
            B_max = np.max([B_max, np.max(np.abs(Zernike_B[:,0]))])

        # the outer boundary
        diff_B = np.abs(Zernike_B[:,-1]-master_B[:,-1])
        diff_B_max = np.max([diff_B_max, np.max(diff_B)])
        B_max = np.max([B_max, np.max(np.abs(Zernike_B[:,-1]))])

    # evaluate error
    diff_B_max = diff_B_max 

    if (diff_B_max < rtol):
        print(key, " matches, diff", diff_B_max)
    else:
        print(key, " doesn't match, diff", diff_B_max)

## 1. Test Lconstraint=0,1,2 cases, fixed boundary
### 1.1 Five volume slab with Lconstraint=0
The example here uses Lrad=16. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.

__We only compare `Bp` ($B_z$) for slabs and cylinders since the other components have different definitions in the master vs Zernike branch.__

_It takes approximately a few seconds to complete the runs._

In [5]:
input_file_name='InputFiles/TestCases/G1V05L0Fi.001.sp'
result = run_case(input_file_name, ncpus=2)

check_interface(result, key='Rbc')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.1102230246251565e-16
Bp  matches, diff 5.551115123125783e-17


### 1.2 Three volume slab with Lconstraint=2
The example here uses Lrad=16. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.
This case is used in ci.

_It takes approximately a few seconds to complete the runs._

In [6]:
input_file_name='InputFiles/TestCases/G1V03L2Fi.002.sp'
result = run_case(input_file_name, ncpus=2)

check_interface(result, key='Rbc')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.7763568394002505e-15
Bp  matches, diff 6.938893903907228e-18


### 1.3 32 volume cylinder with Lconstraint=1
The example here uses Lrad=12. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.
This case is used in ci.

_It takes approximately a few seconds to complete the runs._

In [8]:
input_file_name='ci/G2V32L1Fi/G2V32L1Fi.001.sp'
result = run_case(input_file_name, ncpus=2)

check_interface(result, key='Rbc')
check_field_on_interfaces(result, key='Bp')

Initial guess of the interface geometry ignored: line  102
Initial guess of the interface geometry ignored: line  102
SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 4.440892098500626e-16
Bp  matches, diff 1.4432899320127035e-15


### 1.4 Two-volume rotating ellipse case with Lconstraint=1
We force it to have Lrad=16 in the first volume. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.

_It takes approximately 5 mins to complete the runs. (10s for Zernike, 5 mins for master)_

In [10]:
input_file_name='InputFiles/TestCases/G3V02L1Fi.001.sp'
result = run_case(input_file_name, ncpus=2, LradZernike=16, Lradmaster=12)

check_interface(result, key='Rbc')
check_interface(result, key='Zbs')
check_field_on_interfaces(result, key='BR')
check_field_on_interfaces(result, key='BZ')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 2.6290081223123707e-13
Zbs  matches, diff 1.3888890038060708e-13
BR  matches, diff 3.728614639264549e-14
BZ  matches, diff 3.0107860649053464e-14
Bp  matches, diff 6.50868248186498e-15


### 1.5 Single-volume W7X OP1.1 fixed boundary

The magnetic field on the plasma boundary is compared, tolerance set to 1e-11 (further increase resolution will lead to ill-conditioning in the master branch)

_It takes approximately a minute to complete the runs. (5s for Zernike, 30s for master)_

In [11]:
input_file_name='InputFiles/TestCases/G3V01L0Fi.002.sp'
result = run_case(input_file_name, ncpus=1, LradZernike=32, Lradmaster=14, Mpol=8, Ntor=8)

check_field_on_interfaces(result, key='BR',rtol=1e-11)
check_field_on_interfaces(result, key='BZ',rtol=1e-11)
check_field_on_interfaces(result, key='Bp',rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
BR  matches, diff 4.003131159890927e-12
BZ  matches, diff 4.488409643954583e-12
Bp  matches, diff 2.938205234670477e-13


## 2. Test Lconstraint=0,1,2 cases, free boundary
### 2.1 Three-volume free-boundary vacuum case without stellarator symmetry, FOCUS coil
Section 4.1 and Figure 1 in Hudson PPCF 2020 free-boundary SPEC paper

_It takes approximately 10 mins to complete the runs. (10s for Zernike, 10mins for master)_

In [12]:
input_file_name='ci/toroidal_freeboundary_vacuum/G3V02L0Fr.sp'
result = run_case(input_file_name, ncpus=3, LradZernike=24, Lradmaster=14)

check_interface(result, key='Rbc',rtol=1e-11)
check_interface(result, key='Zbs',rtol=1e-11)
check_interface(result, key='Rbs',rtol=1e-11)
check_interface(result, key='Zbc',rtol=1e-11)
check_field_on_interfaces(result, key='BR',rtol=1e-11)
check_field_on_interfaces(result, key='BZ',rtol=1e-11)
check_field_on_interfaces(result, key='Bp',rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.421533247017437e-13
Zbs  matches, diff 1.2264096530857638e-13
Rbs  matches, diff 4.187157687089195e-13
Zbc  matches, diff 3.416257348624191e-13
BR  matches, diff 8.243614124658905e-13
BZ  matches, diff 8.721582483994794e-13
Bp  matches, diff 1.2521095271722515e-12


### 2.2 Free-boundary case with Lconstraint=1, Nvol=8, provided by A. Baillod

In this case we have to allow lower error (1e-11) tolerance between the two branches since further increasing `Lrad` for the master branch will lead to ill-conditioning.

_It takes approximately 30s to complete the runs. (10s for Zernike, 20s for master)_

__Important: Interestingly, with intel-17 compiler this case may fail. But it is fine with other compilers (including a different version of Intel). Please let Zhisong know if there is a problem with your compiler.__

In [13]:
input_file_name='InputFiles/TestCases/G3V08L1Fr.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=3, LradZernike=22, Lradmaster=9, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-11)
check_interface(result, key='Zbs', rtol=1e-11)
check_field_on_interfaces(result, key='BR', rtol=1e-11)
check_field_on_interfaces(result, key='BZ', rtol=1e-11)
check_field_on_interfaces(result, key='Bp', rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.071143174158351e-12
Zbs  matches, diff 1.7132961716015416e-12
BR  matches, diff 2.5223096181137272e-14
BZ  matches, diff 2.5393315922217496e-14
Bp  matches, diff 8.146261443187086e-15


### 2.3 VMEC-SPEC benchmark case, 8 volumes

Section 4.6, Figure 13 and 14 in Hudson PPCF 2020 free-boundary SPEC paper

In this case we reduce `Mpol` to 8 to avoid ill-conditioning. We do not compare to VMEC, but compare the result from the two branches. We have to allow lower error (1e-10) tolerance.

_It takes approximately 3 mins to complete the runs. (1 mins for Zernike, 2 mins for master)_

__Important: Interestingly, with intel-17 compiler this case may fail. But it is fine with other compilers (including a different version of Intel). Please let Zhisong know if there is a problem with your compiler.__

In [14]:
input_file_name='InputFiles/Verification/FreeBoundVMEC/Nv=008.L=12.M=32.n.sp.end'

replace_namelist = list()
replace_namelist.append(('globallist', 'gBntol', 1e-9))
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=3, LradZernike=16, Lradmaster=10, Mpol=8, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-10)
check_interface(result, key='Zbs', rtol=1e-10)
check_field_on_interfaces(result, key='BR', rtol=1e-10)
check_field_on_interfaces(result, key='BZ', rtol=1e-10)
check_field_on_interfaces(result, key='Bp', rtol=1e-10)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 8.82605100116507e-12
Zbs  matches, diff 1.4408696458190207e-11
BR  matches, diff 2.172437577052655e-14
BZ  matches, diff 1.5379190976272383e-14
Bp  matches, diff 2.6346980153135746e-14


## 3. Current constraint
### 3.1-Fixed boundary, Lconstraint=3, Lfindzero=1 test 

Test the Beltrami field and evaluation of the force with the current constraint. The force gradient (Lfindzero=2) is not used here.

_This case takes approximately 2mins to run_



In [15]:

input_file_name = 'InputFiles/Verification/currentconstraint/TestCases_Comparison/G3V02L3Fi.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=2, LradZernike=16, Lradmaster=10, others=replace_namelist)

check_interface(result, key='Rbc')
check_interface(result, key='Zbs')
check_field_on_interfaces(result, key='BR')
check_field_on_interfaces(result, key='BZ')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 2.984279490192421e-13
Zbs  matches, diff 1.294415963304374e-13
BR  matches, diff 6.757094883624859e-14
BZ  matches, diff 4.917594109699053e-14
Bp  matches, diff 1.0436096431476471e-14


### 3.2 Fixed-boundary test in slab geometry with current constraint

Test the current constraint capability in slab geometry

_Expect less than 10s for the Zernike branch to run and about 40s for the master branch to run_

In [18]:
input_file_name = 'InputFiles/TestCases/G1V02L3Fi.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=2, LradZernike=16, Lradmaster=10, Mpol=4, Ntor=2, others=replace_namelist)

check_interface(result, key='Rbc')
check_interface(result, key='Zbs')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.865174681370263e-14
Zbs  matches, diff 0.0
Bp  matches, diff 4.440892098500626e-15


### 3.3 Fixed-boundary test in cylindrical geometry with current constraint

Test the current constraint capability in cylindrical geometry

_Expect the Zernike branch to run in about 20s and the master branch in about the same_

In [21]:
input_file_name = 'InputFiles/TestCases/G2V32L3Fi.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=4, LradZernike=18, Lradmaster=14, others=replace_namelist)

check_interface(result, key='Rbc')
check_interface(result, key='Zbs')
check_field_on_interfaces(result, key='Bp')

Initial guess of the interface geometry ignored: line  105
Initial guess of the interface geometry ignored: line  104
SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 9.020562075079397e-16
Zbs  matches, diff 0.0
Bp  matches, diff 2.1094237467877974e-15


### 3.4 Free-boundary test in toroidal geometry with current constraint

Test the current constraint capability in toroidal geometry This is rather long to run due to the low value of _gBntol_. For some reasons, the master branch appears to be faster than the Zernike branch on this case.

The tolerance on RBc and ZBs is set to 1E-10 which might seem large. This can be lowered at the cost of lowering gBntol, which increases drastically the computation time. This value has thus been set for the sake of a rapid test.

__Important: The master branch contains a bug that may lead to array out of bound error. If it causes segmentation fault on your machine, you will need to apply the following modification.__

`master` branch, `dforce.f90`, Line 156, change the size of `ipiv` to
```fortran
INTEGER              :: ipiv(1:Mvol)
```
Then the code should work fine.

_Expect ~12min for Zernike (36 free-boundary iterations) and 10min for master branch_

In [29]:
input_file_name = 'InputFiles/TestCases/G3V08L3Fr.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))
replace_namelist.append(('globallist', 'gBNtol', 1E-10))
replace_namelist.append(('globallist', 'gBnbld', 0.2))
replace_namelist.append(('globallist', 'Lfindzero', 2))
replace_namelist.append(('globallist', 'mfreeits', 100))

result = run_case(input_file_name, ncpus=3, LradZernike=18, Lradmaster=12, Mpol=2, Ntor=1, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-10)
check_interface(result, key='Zbs', rtol=1e-10)
check_field_on_interfaces(result, key='BR')
check_field_on_interfaces(result, key='BZ')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.866062859789963e-11
Zbs  matches, diff 3.1743774275838632e-12
BR  matches, diff 4.339280670895285e-14
BZ  matches, diff 2.2952993672387123e-14
Bp  matches, diff 3.070946275052222e-13
